In [1]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent


import json
import glob





print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

2025-01-01 05:05:53.544392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735704353.580981  156117 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735704353.592411  156117 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-01 05:05:53.628819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.18.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

DATA_FS251 = './data/iFood_2019'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

MODELS = './models/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [3]:
############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

,file_name,class_num
0,train_101733.jpg,211
1,train_101734.jpg,211
2,train_101735.jpg,211


In [4]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 20
BATCH_SIZE = 1              # 8 fit into GPU RAM, 64 fit into system RAM
RESIZE_TO = (256, 256)
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

In [5]:
############################################################################################
# FUNCTIONS, DEFFINITIONS
############################################################################################

class CRelu(tf.keras.Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis 
        super(CRelu, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CRelu, self).build(input_shape)

    def call(self, x):
        x = tf.nn.crelu(x, axis=self.axis)
        return x

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = output_shape[-1] * 2
        output_shape = tuple(output_shape)
        return output_shape

    def get_config(self, input_shape):
        config = {'axis': self.axis, }
        base_config = super(CReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [6]:
############################################################################################
# import image files
############################################################################################

train_pics, val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='both',
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format='channels_last',
    verbose=True
)
test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=None,
    subset=None,
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format='channels_last',
    verbose=True
)
#true_test_pics = tf.keras.preprocessing.image_dataset_from_directory(
#    TEST_PICS_PATH,
#    labels=None,
#    label_mode=None,
#    class_names=None,
#    color_mode='rgb',
#    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
#    shuffle=True,
#    seed=SEED,
#    validation_split=None,
#    subset=None,
#    interpolation='bilinear',
#    follow_links=False,
#    crop_to_aspect_ratio=False,
#    pad_to_aspect_ratio=True,
#    data_format='channels_last',
#    verbose=True
#)

Found 19368 files belonging to 51 classes.
Using 15495 files for training.
Using 3873 files for validation.


I0000 00:00:1735704359.232138  156117 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5456 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1


Found 2607 files belonging to 51 classes.


In [7]:
train_pics.class_names


['000-macaron',
 '001-beignet',
 '002-cruller',
 '003-cockle',
 '004-samosa',
 '005-tiramisu',
 '006-tostada',
 '007-moussaka',
 '008-dumpling',
 '009-sashimi',
 '010-knish',
 '011-croquette',
 '012-couscous',
 '013-porridge',
 '014-stuffed_cabbage',
 '015-seaweed_salad',
 '016-chow_mein',
 '017-rigatoni',
 '018-beef_tartare',
 '019-cannoli',
 '020-foie_gras',
 '021-cupcake',
 '022-osso_buco',
 '023-pad_thai',
 '024-poutine',
 '025-ramen',
 '026-pulled_pork_sandwich',
 '027-bibimbap',
 '028-chicken_kiev',
 '029-apple_pie',
 '030-risotto',
 '031-fruitcake',
 '032-chop_suey',
 '033-haggis',
 '034-scrambled_eggs',
 '035-frittata',
 '036-scampi',
 '037-sushi',
 '038-orzo',
 '039-fritter',
 '040-nacho',
 '041-beef_stroganoff',
 '042-beef_wellington',
 '043-spring_roll',
 '044-savarin',
 '045-crayfish',
 '046-soufflé',
 '047-adobo',
 '048-streusel',
 '049-deviled_egg',
 '050-escargot']

In [8]:
#plt.figure(figsize=(15, 15))
#for images, labels in train_08_pics.take(1):
#    for i in range(9):
#        ax = plt.subplot(3, 3, i + 1)
#        plt.imshow(images[i].numpy().astype("uint8"))
#        plt.title(class_names[i])
#        plt.axis("off")

In [ ]:
############################################################################################
# DEFINING THE MODEL
############################################################################################
tf.keras.backend.clear_session()
model_supclass = tf.keras.models.Sequential(layers=[
# Zero's layer - resizing:
        tf.keras.layers.InputLayer(shape=(256,256,3)),
        tf.keras.layers.Resizing(height = 256,width = 256,interpolation='bilinear',crop_to_aspect_ratio=False,pad_to_aspect_ratio=True,fill_mode='constant',fill_value=0.0,data_format=None),
        tf.keras.layers.Rescaling(1./255),
# First conv. layers set 256:
        tf.keras.layers.Conv2D(filters=32,kernel_size = (9,9),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.Conv2D(filters=32,kernel_size = (9,9),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.Conv2D(filters=32,kernel_size = (9,9),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.BatchNormalization(),
#        tf.keras.layers.Normalization(),
# Second conv. layers set 256>128:
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2,padding='valid',data_format=None,name='256_to_128_MaxPooling2D'),
        tf.keras.layers.Conv2D(filters=64,kernel_size = (7,7),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.Conv2D(filters=64,kernel_size = (7,7),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.Conv2D(filters=64,kernel_size = (7,7),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.BatchNormalization(),
# Second conv. layers set 128->64:
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2,padding='valid',data_format=None,name='128_to_64_MaxPooling2D'),
        tf.keras.layers.Conv2D(filters=128,kernel_size = (5,5),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.Conv2D(filters=128,kernel_size = (5,5),strides=(1, 1),padding='same',data_format=None,dilation_rate=(1, 1),groups=1,activation='leaky_relu',use_bias=True,kernel_initializer='glorot_uniform',bias_initializer='zeros',kernel_regularizer=None,bias_regularizer=None,activity_regularizer=None,kernel_constraint=None,bias_constraint=None),
        tf.keras.layers.BatchNormalization(),
# 3rd conv. layers set 64->32:
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2,padding='valid',data_format=None,name='64_to_32_MaxPooling2D'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=1024, kernel_initializer = 'uniform', activation='leaky_relu'),
#        tf.keras.layers.Dropout(0.1),
#        tf.keras.layers.Dense(units=1024, kernel_initializer = 'uniform', activation='elu'),
        tf.keras.layers.Dense(units=1024,activation='leaky_relu'),
        tf.keras.layers.Dense(units=51,activation='softmax')
])

model_supclass.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resizing (Resizing)             │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 3)    │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 256)  │        62,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 256)  │     5,308,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 256)  │     5,308,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 128, 128, 256)  │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 256)  │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 256)  │     3,211,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 256)    │     1,638,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 256)    │     1,638,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 256)    │     1,638,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 32, 32, 256)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 16, 16, 512)    │             0 │
│ (AveragePooling2D)              │                        │             

 Total params: 153,371,199 (585.06 MB)

 Trainable params: 153,362,489 (585.03 MB)

 Non-trainable params: 8,710 (34.02 KB)

In [10]:
############################################################################################
# COMPILING
############################################################################################
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.00005, #0.000005
    decay_steps=STEPS_PER_EPOCH*100, #100
    decay_rate=1,
    staircase=False)

model_supclass.compile(
#        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),                     
#        optimizer = tf.keras.optimizers.Adam(),                                     #
#        optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.5),                 #3            
#        optimizer = tf.keras.optimizers.Adagrad(),                                             
#        optimizer = tf.keras.optimizers.Adam(0.005),
#        optimizer = tf.keras.optimizers.Nadam(),
#       optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.5),                 #1
        optimizer = tf.keras.optimizers.Nadam(learning_rate=0.5),                  #2
        loss = tf.keras.losses.CategoricalCrossentropy(),
#        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
#        metrics = [tf.keras.metrics.CategoricalCrossentropy(from_logits=False, label_smoothing=0.2, axis=-1),
#                   tf.keras.metrics.F1Score(average=None, threshold=0.5, dtype=None),
#                   tf.keras.metrics.CategoricalAccuracy()]
        metrics = [tf.keras.metrics.Accuracy()]
#        metrics = [tf.keras.metrics.F1Score(average='micro', threshold=0.5),tf.keras.metrics.CategoricalAccuracy()]
        )

In [11]:
############################################################################################
# FITTING
############################################################################################
with tf.device('/gpu:0'):
    model_history = model_supclass.fit(
        train_pics,
        epochs = EPOCHS, 
        batch_size = BATCH_SIZE,
#        callbacks = tf.keras.callbacks.EarlyStopping(
#                            monitor='val_loss',
#                            min_delta=0.01,
#                            patience=1,
#                            verbose=1,
#                            mode='auto',
#                            baseline=None,
#                            restore_best_weights=True,
#                            start_from_epoch=0),
        validation_data = val_pics
#        validation_split = 0.1
    )

Epoch 1/20


I0000 00:00:1735704381.630205  156228 service.cc:148] XLA service 0x77a894002020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735704381.630252  156228 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 with Max-Q Design, Compute Capability 6.1
2025-01-01 05:06:22.355247: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1735704383.583543  156228 cuda_dnn.cc:529] Loaded cuDNN version 90600
2025-01-01 05:06:27.002125: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng35{k2=4,k3=0} for conv (f32[1,256,256,256]{3,2,1,0}, u8[0]{0}) custom-call(f32[1,256,256,256]{3,2,1,0}, f32[256,256,9,9]{3,2,1,0}, f32[256]{0}), window={size=9x9 pad=4_4x4_4}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"

 1270/15495 ━━━━━━━━━━━━━━━━━━━━ 8:08:16 2s/step - accuracy: 0.0000e+00 - loss: 4.8409

KeyboardInterrupt: 

In [ ]:
############################################################################################
# SAVING MODEL
############################################################################################

os.makedirs( os.path.dirname(MODELS), exist_ok=True)
model_supclass.save(MODELS + "251_dima_superclasses_CUSTOM_v1.5_FOLDERFLOW_OHE_sigm.keras")
model_supclass_loaded = load_model(MODELS + "251_dima_superclasses_CUSTOM_v1.5_FOLDERFLOW_OHE_sigm.keras")
model_supclass_loaded.summary()



In [ ]:
############################################################################################
# EVALUATION
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['accuracy'],label='train_accuracy')
plt.plot(model_history.history['val__accuracy'],label='val_accuracy')
plt.xlabel('epochs')
plt.ylabel('categorical_accuracy')
plt.legend()

# Print out the score
score = model_supclass_loaded.evaluate(val_pics , batch_size=1, verbose=1)
print(score, model_supclass_loaded.metrics_names)

In [ ]:
############################################################################################
# PREDICTION
############################################################################################

y_test_pred = np.array([])
y_test =  np.array([])
for x, y in test_pics:
    y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass.predict(x, verbose=0), axis = -1)])
    y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])

In [ ]:
############################################################################################
# PREDICTION - CONFMATRIX
############################################################################################

from sklearn.metrics import ConfusionMatrixDisplay
plt.figure(figsize = (30,30))
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true= y_test, y_pred=y_test_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.unique(y_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()




